In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from fontTools.ttLib import TTFont
from PIL import Image, ImageDraw, ImageFont
from tensorflow.python.lib.io.file_io import FileIO

2024-02-02 14:01:22.353667: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 14:01:22.353716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 14:01:22.355321: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 14:01:22.364660: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def get_existing_chars(filename):
    ttfont = TTFont(filename)
    return list({chr(key) for table in ttfont['cmap'].tables for key in table.cmap.keys()})

In [3]:
def get_image(font, char):
    img = Image.new('L', (500,500), 255)
    
    draw = ImageDraw.Draw(img)
    draw.text((0,0), char, font=font)
    
    npimg = 255 - np.array(img)
    wmin = npimg.sum(0).nonzero()[0].min()
    wmax = npimg.sum(0).nonzero()[0].max()
    hmin = npimg.sum(1).nonzero()[0].min()
    hmax = npimg.sum(1).nonzero()[0].max()
    npimg = 255 - npimg[hmin:hmax+1,wmin:wmax+1]
    return Image.fromarray(npimg)
    # return npimg

In [4]:
fd = "ai_font/data/r40202/ttfs/"
ttfs = [(fd+"twice/"+f,"twice", f.split(".ttf")[0]) for f in os.listdir(fd+"twice")]
ttfs += [(fd+"uhbee/"+f,"uhbee", f.split(".ttf")[0]) for f in os.listdir(fd+"uhbee")]

In [5]:
png_fd = "ai_font/data/r40202/pngs/"

In [6]:
pbar = tqdm(range(1,len(ttfs)))
for i in pbar:
    ttf = ttfs[i]
    filename = ttf[0]
    font = ImageFont.truetype(filename,255)
    existing_chars = get_existing_chars(filename)
    chardf = pd.DataFrame(existing_chars)
    chardf.to_parquet(png_fd+"%s__%s.parquet"%(ttf[1],ttf[2]))
    for i in range(len(chardf)):
        char = chardf.iloc[i].values[0]
        try:
            img = get_image(font, char)
            with open(png_fd+'%s__%s__%s.png'%(ttf[1],ttf[2],str(i)), "wb") as f:
                img.save(f, "PNG")
        except:
            # print(ttf[1], ttf[2], char)
            pass
    if i % 2 == 0:
        pbar.set_postfix(n_files=str(len(os.listdir(png_fd))))

100%|██████████| 171/171 [25:35<00:00,  8.98s/it, n_files=523414]
